In [ ]:
# new approach, try to crawl through all athletes!

# I have this nice big list of results pages now in the ResultsLinksTable
# (Check for events with no entries there...)
# maybe visit every one of those pages, then go to the athlete pages
# and look at their related athletes!

# next step: go to the page of every sport (already have that info)
# get the "featured athletes", and start with those as seeds

# when I start actually parsing results, I should have periodic dumps to file
# so the list of results doesn't grow out of control
# like, maybe every 100 results or something

# one of the errors that came up was from spaces in the link. Need to use urllib2.quote()

import logging
reload(logging)
logger = logging.getLogger(__name__)

In [ ]:
import urllib2
import re
import json
from bs4 import BeautifulSoup
import sys

In [ ]:
logging.basicConfig(filename='./AthleteCrawler.log',filemode='w',level=logging.DEBUG)

In [ ]:
# base url for olympics site
baseUrl = "https://www.olympic.org"

# seed with Michael Phelps
athletesToVisit = []
athletesToVisit.append("/michael-phelps")
log.info("Seeding crawler with ",athletesToVisit)

# visited already
athletes = set()

while len(athletesToVisit) > 0:
    athlete = athletesToVisit.pop()
    try:
        # get the html
        html = urllib2.urlopen(baseUrl+"/"+athlete).read()

        # soup it up
        athleteSoup = BeautifulSoup(html)

        # first get the nationality of this athlete
        profileTag = athleteSoup.find("div",class_="profile-row")
        nat = profileTag.find("span").get_text()

        # get the id card
        idcard = athleteSoup.find("section",class_=re.compile("id-card.*")) 
        idcardLinks = idcard.find_all("a")
        sport = idcardLinks[0].string
        county = idcardLinks[1].string 
        born = idcard.find_all("div",class_="text-box")[2].get_text().split("\n")[-1].strip()

        athletes.add(athlete)
        logging.info("Visited " + athlete + " - " + nat + " - " + sport)


        # find the related athlete section
        relatedSection = athleteSoup.find("section",class_="related-athletes")

        # get the links from that section, holding the events
        nextLinks = relatedSection.find_all("a",itemprop="url")
        for link in nextLinks:
            nextAthlete = link['href']
            if (not nextAthlete in athletes) and (not nextAthlete in athletesToVisit):
                athletesToVisit.append(nextAthlete)
    except Exception as err:
        logging.warning("Problem with " + athlete + ": " + format(err))
log.info("Athlete crawl complete")